In [ ]:
"""MgH+ Polaritonic Potential Energy Surfaces"""

__authors__ = ["Jonathan J. Foley"]
__email__   = ["jfoley19@uncc.edu"]
__credits__ = ["Jonathan J. Foley"]
__copyright__ = "(c) 2008-2020, The Psi4Education Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-02-11"

## Polaritonic chemistry for a simple diatomic molecule strongly interacting with light

#### Learning Objectives

- Students will be able to define a molecular polariton

- Students will be able to explain how forming molecular polaritons can modify chemical structure and reactivity

- Students will be able to use quantum chemistry calculations to parameterize a Rabi model Hamiltonian for a molecular polariton

- Students will be able to construct polaritonic potential energy surfaces using the Rabi model

- Students will be able to test a quantitative relationship between the coupling strength and splitting between polaritonic potential energy surfaces

#### Supplementary learning objective

- Students will be able to apply the rules of 2nd quantized operators to derive the Rabi Hamiltonian matrix

#### Background
Nanostructured materials can be designed to strongly confine light down to nanoscale dimensions, which 
greatly enhances the energy density of light and increases its interaction with molecular excitations.
When molecules through their excitations *strongly* interact with light, new quantum states, called polariton states, emerge that have
properties of light and of matter.  

Consider the image below that shows the MgH+ molecule strongly 
interacting with a nanoconfined photon trapped between two parallel mirrors (aka a cavity). This cavity 
confines a photon with electric field $\vec{E}$ aligned with the transition dipole moment $\vec{\mu}_{i\rightarrow f}$  connecting the ground to first singlet excited state of MgH+.  When the energy of the trapped photon ($\hbar \omega$) matches this excitation energy in a particular region of the MgH+ potential energy surface, energy can be 
rapidly exchanged between the molecular excitation and the cavity.  We can think of this exchange as cylcing between  the molecule absorbing the trapped photon and becoming excited, and relaxing back to the ground-state and emitting the a photon which becomes trapped in the cavity.  When this cycling is rapid, as is the case when the interaction 
is strong, the cycling of energy makes it impossible to distinguish between the quantum state of the excited molecule and the quantum state of the photon localized in the cavity.  Rather, we think of this situation as a hybrid state that is part light part matter, and we call this state a polariton.  One fascinating feature of molecular polaritons
is that their potential energy surfaces may be dramatically different than the potential energy surfaces of their
molecular constituents.  Consequently, new and different chemical reactivity may be realized by putting molecules in cavities!  This notebook will explore the reshaping of the potential energy surfaces of the simple diatomic
MgH+ by placing it in a cavity.

<img src="data/MgH_polariton.png" width=300 height=300 />
 



##### What is a molecular polariton?

A *molecular polariton* can be understood as quantum state that mixes the quantum states of a photon and the quantum states of a molecule.  The two criteria for forming a molecular polariton are that

1. The photon energy should match the transition energy between molecular quantum states

2. The energy scale of the interaction between the photon and the molecular transition should be large compared to dissipation energy scales in the individual molecular and photonic systems




**Question 1** Let's consider the first criteria for electronic transitions in the MgH+ system.  The the relevant electronic transition (ground to first singlet excited state) energy is on the order of 4 eV.  What photon wavelength can be used to form a molecular polariton with this electronic transition assuming a transition energy of 4 eV?

*Hint:* $ E = \hbar \omega $ and $ \lambda = \frac{2\pi c}{\omega} $  

In [13]:
transition_energy = 4.0 # g -> e transition energy in eV

hbar_eV_s = 6.582119569e-16 # -> hbar in eV * s

c_SI = 299792458 # -> speed of light in m / s

m_to_nm = 1e9 # -> conversion between meters and nanometers

# ==> Write code to compute wavelength in meters here <==
omega_SI = transition_energy / hbar_eV_s

lambda_SI = np.pi * 2 * c_SI / omega_SI

# ==> Write code to convert wavelength from meters to nanometers here... store in variable lambda_nm <==
lambda_nm = lambda_SI * m_to_nm

print(f'The photon wavelength is {lambda_nm:.3f} nm')

The photon wavelength is 309.960 nm


Let's consider the second criteria that the interaction energy scale must be large compared to the dissipation energy scales of the individual molecular and photonic system. Specifically, we have the criteria 
$$ \hbar g > \hbar \frac{\gamma_p + \gamma_m}{4} $$ 
where $\hbar g$ represents the interaction energy scale, $\hbar \gamma_p$ represents the dissipation energy scale of the photon and $\hbar \gamma_m$ represents the dissipation energy scale of the molecular excitation.  These $\gamma$ terms represent dissipation rates, and can be interpreted as the inverse of the lifetime of the occupation of the photon in the cavity in the case of $\gamma_p$ or the inverse of the lifetime of an electronic excitation in the case of $\gamma_m$.  Typically, $\gamma_p >> \gamma_m$ so we will assume that $\gamma_p + \gamma_m \approx \gamma_p $ for the analysis that follows.

We can define $g$ as follows:

$$ g = \sqrt{ \frac{\omega}{2}} \lambda \cdot \mu_{ge}, $$
where the fundamental coupling strength $\lambda$ is defined as 
$$ \lambda = \sqrt{\frac{1}{\epsilon_0 V}}, $$
$\mu_{eg}$ is the transition dipole moment associated with the electronic transition, and 
$V$ represents the volume of the cavity that confines the photon (and molecule).  Here we can
see that for a given value of the transition dipole moment, increasing the confinement of the photon by decreasing $V$ increases the interaction energy scale by increasing the fundamental coupling strength $\lambda$. 

**Question 2**
If we can assume that (in atomic units) $\mu_{ge} = 1$, $\omega = 0.1469$, and $\gamma_p = 0.0016$, what is the value of $V$ required to satisfy $g = \frac{\gamma_p}{4}$?  
*Hint:* In atomic units, $\frac{1}{\epsilon_0} = 4 \pi$.

In [18]:
mu_ge_au = 1 # -> transition dipole moment in atomic units 

gamma_p_au = 0.0016 # -> photon dissipation rate in atomic units

omega_au = 0.1469 # -> photon frequency in atomic units

au_to_nm = 0.0529 # -> conversion factor for atomic units of length to nanometers

# ==> Write code to compute V in atomic units here <== 
V_au = 4 * np.pi / (gamma_p_au / (4 * mu_ge_au * np.sqrt(omega_au / 2) ) ) ** 2

# ==> Convert volume in atomic units to volume in nm^3 <==
V_nm = V_au * au_to_nm ** 3

print(f'The cavity volume is {V_nm:.3f} nm^3')

The cavity volume is 853.982 nm^3


#### Model system
We will consider the diatomic cation MgH+ in a cavity chosen such that it can trap a photon with a 
frequency of $\hbar \omega = 4.3$ eV. 
We will compute the singlet ground state and first singlet excited state potential energy surfaces along the
Mg-H+ stretch coordinate $R$ using density functional theory (DFT) and time-dependent density functional theory (TDDFT), respectively.  DFT provides an efficient method for computing the ground-state energy of a quantum mechanical system using the electron density (rather than the wavefunction) as the primary variable, and TDDFT provides an efficient method for estimating the energies and properties of excited-states starting from the DFT ground state.    We denote the ground 
state ket as $|g\rangle$ with an associated energy eigenvalue $E_g(R)$, and the excited-state $|e\rangle$
with associated energy eigenvalue $E_e(R)$ where $R$ is the bondlength.  The transition dipole
moment between $|g\rangle$ and $|e\rangle$ will also be computed using TDDFT, yielding $\mu_{ge}(R)$. 
While
the transition dipole moment and the electric field are vector quantities, in this case $\mu_{ge}(R)$ is oriented along the internuclear coordinate ($R$), and so for simplicity we consider an electric field oriented soley along
$R$ as well and treat both quantities as scalars.

We will consider two quantum states for the photon - no photon in the cavity, denoted by $|0\rangle$, and 1 photon in the cavity $|1\rangle$.  The photon has an energy of $\hbar \omega$. At the same time, we will allow for the molecule to be in its electronic ground state $|g\rangle$ or its first electronic excited state $|e\rangle$.  
This leads to the following basis states for the composite system:

$|g,0\rangle \rightarrow$ The molecule is in the ground state and there is no photon in the cavity.

$|g,1\rangle \rightarrow$ The molecule is in the ground state and there is one photon in the cavity.

$|e,0\rangle \rightarrow$ The molecule is in its first electronic excited state and there is no photon in the cavity.

We will neglect the $|e,1\rangle$ state for simplicity, and if desired, it can be shown that its inclusion does not impact the results.

In this 3-state basis, we will model this system with a generalized Rabi Hamiltonian that can be written as:
\begin{equation}
    {\bf H} =
    \begin{pmatrix}
    E_g(R)  & 0 & 0 \\
    0 & E_g(R) + \hbar \omega &  \lambda \cdot \mu_{ge}(R) \\
    0 &  \lambda \cdot \mu_{ge}(R) & E_e(R)
    \end{pmatrix}.
\end{equation}
where $E_g(R)$ denotes the ground-state potential energy surface, i.e. the energy of state $|g\rangle$ at bondlength $R$, $E_e(R)$ denotes the potential energy surface of the first excited state (energy of state $|e\rangle$ at bondlength $R$), and $\mu_{ge}(R)$ denotes the transition dipole moment between state $|g\rangle$ and $|e\rangle$.  Applying second quantized operators to derive this Hamiltonian matrix is supplementary learning outcome; interested students are directed to [Derivation](#Derivation) cell for the derivation.

##### Procedure
We will build and diagonalize this matrix across a range of bondlength values ($R$), and the 
resulting eigenvalues will comprise the polaritonic potential energy surfaces.  The resulting 
eigenvectors will comprise the polaritonic energy eigenstates.  We will utilize the `psi4` package to compute
$E_g(R)$, $E_e(R)$, and $\mu_{ge}(R)$ to be used in the model Hamiltonian above.

We will import psi4 to have access to the DFT engine for computation of ground-state properties, and the scf_response library of psi4 to have access to TDDFT methods for obtaining excited-state properties.  We will also import numpy and scipy libraries to help with constructing and diagonalizing the Rabi Hamiltonian.

In [21]:
### Import libraries to be used throughout
# basic psi4 library
import psi4
# numpy
import numpy as np
# scipy
from scipy.interpolate import InterpolatedUnivariateSpline
# linear algebra package from numpy
from numpy import linalg as LA
# time-dependent scf library from psi4 for computing excited states and transition dipole moments
from psi4.driver.procrouting.response.scf_response import tdscf_excitations
#from matplotlib import pyplot as plt

To compute $E_g(R)$, $E_e(R)$, and $\mu_{ge}(R)$, we will use DFT/TDDFT at the B3LYP/cc-pVDZ level for 25 different geometries from $R = 1.1$ to $R = 3.5$ Angstroms.

The scan of different $R$ values uses similar syntax as has been demonstrated in other Psi4Education labs, see for example the lab on the [calculation of spectroscopic constants](https://github.com/Psi4Education/psi4education/blob/master/labs/spectroscopic_constants/spectroscopic_constants_student.ipynb).

Extracting the excitation energy and the transition dipole moment requires that we run the `tdscf_excitations` method from the psi4 package, which we have explicitly imported.  This method returns a dictionary containing different information about the excited-state energies and properties. 

##### The tdscf_excitations method and dictionary
The syntax for computing singlet excited-state properties with the tdscf_excitations method is as follows:

`res = tdscf_excitations(wfn, states=n, triplets='None')`

where `wfn` is the wavefunction object that can be obtained from a prior ground-state calculation, `states=n` specifies the number `n` of excited-states you wish to compute, `triplets='None'` specifies that only singlet exited states will be obtained, and `res` is the name specified for the nested dictionary that the method returns to store the excited-state information.  

Dictionaries are used to store data values in key:value pairs, and nested dictionaries have multiple dictionaries inside a dictionary.  This construction is useful for the excited-state calculations because there are many excited states, each having properties that can be specified by the same key.   

We are interested in two keys for our calculation.
The key `"EXCITATION ENERGY"` can be used to access the excitation energy from the ground state to a given excited state obtained from the calculation, and the key `"ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"` can be used to access the transition dipole moments associated with the computed states (ground-to-excited state only).  Because we are interested in these properties only for the first excited state, we will access them as follows:

`res[0]["EXCITATION ENERGY"]`: $E_e(R) - E_g(R)$

`res[0]["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"]`: $\mu_{ge}(R)$.

To see all of the keys of the nested dictionary called `res`, you can type `res[0].keys()`.   For a more general discussion of nested dictionaries, see [here](https://www.programiz.com/python-programming/nested-dictionary).


In [22]:
# set basis
psi4.set_options({
    'basis':'cc-pVDZ'
})

# set the number of electronic states... this is the ground state + n_states more
# we will get 1 excited-state
n_states = 1

# set the number of bond lengths to compute the stretch along
n_geoms = 25

# initialize geometry list
geoms = []

# initialize energy list... note
# there will be the ground state energy + n_states excited state energies
Es = np.zeros((n_states+1, n_geoms))

# initialize z-component of transition dipole list
mu_z = np.zeros((n_states, n_geoms))

# generate bond lengths
rs = []
for i in range(0,n_geoms):
    rs.append(1.1 + i*0.1)

# loop over bond lengths
ctr = 0
for i in rs:
    # generate the MgH+ molecule using a z-matrix and set the Mg-H+ bond length
    mol = psi4.geometry("""
    Mg
    H 1 """ + str(i) + """
    symmetry c1
    1 1
    """)
    # save the geometry
    geoms.append(mol.geometry().to_array())
    psi4.set_options({
    'save_jk': True,
    })  
   
    # calculate and save the ground-state energy and wavefunction
    e, wfn = psi4.energy("b3lyp/cc-pVDZ", return_wfn=True, molecule=mol)
    
    # calculate the excited-state energies and save them to a dictionary called 'res'
    # !!! Insert a markdown section that explains the syntax of the tdscf_excitations method
    #     pay particular attention to explaining the dictionary because it is not
    #     a requisite python skill...
    #     also tell the user how to see all the keys by printing the entire dictionary!
    res = tdscf_excitations(wfn, states=n_states, triplets = "NONE")
    
    # parse the excitation energies from the 'res' dictionary 
    delta_e = [r["EXCITATION ENERGY"] for r in res]
    
    # parse the transition dipole moment from the 'res' dictionary
    mu = [r["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"] for r in res]
    Es[0,ctr] = e
    
    # store the results to the respective arrays
    for j in range(0, n_states):
        Es[j+1,ctr] = e + delta_e[j]
        # !!! explain we only want the z-component which is index 2!
        mu_z[j,ctr] = mu[j][2]
    # print if you want to
    # print(i, Es[:,ctr])
    
    # increment the counter!
    ctr += 1

In [35]:
res = tdscf_excitations(wfn, states=n_states, triplets = "NONE")
print(res[0].keys())

dict_keys(['EXCITATION ENERGY', 'ELECTRIC DIPOLE TRANSITION MOMENT (LEN)', 'OSCILLATOR STRENGTH (LEN)', 'ELECTRIC DIPOLE TRANSITION MOMENT (VEL)', 'OSCILLATOR STRENGTH (VEL)', 'MAGNETIC DIPOLE TRANSITION MOMENT', 'ROTATORY STRENGTH (LEN)', 'ROTATORY STRENGTH (VEL)', 'SYMMETRY', 'SPIN', 'RIGHT EIGENVECTOR ALPHA', 'LEFT EIGENVECTOR ALPHA', 'RIGHT EIGENVECTOR BETA', 'LEFT EIGENVECTOR BETA'])


Now that we have the energies and transition dipole moments, we will plot $E_g(R)$, 
$E_e(R)$, and 
$E_g(R) + \omega$.

In [3]:
plt.plot(rs[:], Es[0,:]*27.211, 'red',label="Eg")
plt.plot(rs[:], Es[1,:]*27.211, 'orange', label="Ee")
plt.plot(rs[:], Es[0,:]*27.211+4.3, 'blue', label="Eg+$\omega$")
plt.xlim(1.1,3.9)
plt.legend()
plt.show()

NameError: name 'plt' is not defined

We will also plot $\mu_{ge}(R)$.

In [ ]:
# add axis labels
plt.plot(rs[:], np.abs(mu_z[0,:]), 'ro-')
plt.xlim(1.1,3.9)
plt.show()

#### Next steps

1. Fit each surface to a spline

2. Define functions that build the Rabi Hamiltonian and extract the eigenstates from them.  For reasons of convenience with the plotting widget, we will create two almost identical functions that extract the lower-polariton and upper-polariton roots separately. 

** NOTE ** In the long-term, make the Rabi Hamiltonian a separate function
that students will write and have the lp and up functions call that function

3.  Launch widget  

In [3]:
# fit all surfaces to a spline
Eg_spline = InterpolatedUnivariateSpline(rs, Es[0,:], k=3)
Ee_spline = InterpolatedUnivariateSpline(rs, Es[1,:], k=3)
mu_spline = InterpolatedUnivariateSpline(rs, np.abs(mu_z[0,:]), k=3 )

In [4]:
# function to generate the lower polariton surface
def lp(amplitude, omega, r_val, g_spline, e_spline, tdm_spline):
    H = np.zeros((3,3))
    lp_vals = np.zeros_like(r_val)
    for i in range(0, len(r_val)):
        if amplitude == 0:
            lp_vals[i] = g_spline(r_val[i]) + omega
        else:
            H[0,0] = g_spline(r_val[i])
            H[1,1] = g_spline(r_val[i]) + omega
            H[1,2] = amplitude * tdm_spline(r_val[i])
            H[2,1] = amplitude * tdm_spline(r_val[i])
            H[2,2] = e_spline(r_val[i])
            vals, vecs = LA.eigh(H)
            lp_vals[i] = vals[1]
        
    return lp_vals


# function to generate the upper-polariton surface
def up(amplitude, omega, r_val, g_spline, e_spline, tdm_spline):
    H = np.zeros((3,3))
    up_vals = np.zeros_like(r_val)
    for i in range(0, len(r_val)):
        if amplitude == 0:
            up_vals[i] = e_spline(r_val[i])
        else:
            H[0,0] = g_spline(r_val[i])
            H[1,1] = g_spline(r_val[i]) + omega
            H[1,2] = amplitude * tdm_spline(r_val[i])
            H[2,1] = amplitude * tdm_spline(r_val[i])
            H[2,2] = e_spline(r_val[i])
            vals, vecs = LA.eigh(H)
            up_vals[i] = vals[2]
        
    return up_vals





In [5]:
om = 4.3/27.211

compute_lp = lp(0.003, om, rs, Eg_spline, Ee_spline, mu_spline)
compute_up = up(0.003, om, rs, Eg_spline, Ee_spline, mu_spline)

#plt.plot(rs, Eg_spline(rs)+om, 'red', label="|g>+$\omega$")
#plt.plot(rs, compute_lp, 'orange', label="|LP>")
#plt.plot(rs, compute_up, 'purple', label="|UP>")
#plt.plot(rs, Ee_spline(rs), 'blue', label="|e>")
#plt.legend()
#plt.show()

In [6]:
%matplotlib widget
import ipympl
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

In [7]:
# Create a slider widget and a widget to hold the plot.
slider = widgets.FloatSlider(description = r'$g$',
                            value = 0,
                            min = 0,
                            max = 5,  
                            step = 1)
# We can use HTML styles to control the appearance of the widget.
plot_widget = widgets.Output(layout = {'width':'100%', 'border': '1px solid black'})

In [8]:

# Turn off interactive mode before creating the 
# plot so it doesn't display too early.
plt.ioff() # Turn off interactive mode
fig, ax = plt.subplots(constrained_layout = True, figsize=[5,4]);
plt.ion() # Turn on interactive mode

#line1, = ax.plot(rs, a_sin(slider.value, rs, a_spline));
#line2, = ax.plot(rs, b_sin(slider.value, rs, b_spline));
line1, = ax.plot(rs, lp(0.00, om, rs, Eg_spline, Ee_spline, mu_spline));
line2, = ax.plot(rs, up(0.00, om, rs, Eg_spline, Ee_spline, mu_spline));
ax.set_ylabel(r'$A\sin(x)$')
ax.set_xlabel(r'$x$')

# force the figure to display in the plot_widget
with plot_widget:
    display(fig.canvas)


In [9]:
# Create a function to re-draw the plot using the value of the slider. It is 
# faster to change the y-data but you can erase the plot and draw a new one.
def update(value):
    '''We can use `slider.value` or `value.new` to get the new slider value.'''
    with plot_widget:
        line1.set_ydata(lp(slider.value*0.001, om, rs, Eg_spline, Ee_spline, mu_spline))
        line2.set_ydata(up(slider.value*0.001, om, rs, Eg_spline, Ee_spline, mu_spline))
        fig.canvas.draw()

# Set an observer to call `update` whenever the value changes.
slider.observe(update)

In [10]:
widgets.VBox([plot_widget, slider])

In [ ]:
# conversion from atomic units of field strength 
# to GV / m as is used in the Figure 3 of 
# J. Chem. Phys. 153 234304 (2020)
Efield_SI_to_au = 5.14220674763e-11

# Medium electric field strength from paper JCP paper in SI units
E_field_medium = 3e12

# electric field in atomic units
#E_au = E_field_medium * Efield_SI_to_au
E_au = 0.003

Htot = np.zeros((3,3))

''' Polaritonic Hamiltonian will have the following structure

    | E_g(r)               0                        0            |
    |            E_g(r) + hbar * omega              E_au*mu(r)   |
    | 0                  E_au*mu(r)                   E_e(r)     |
    
'''

pl_1 = np.zeros_like(rs)
pl_2 = np.zeros_like(rs)
pl_3 = np.zeros_like(rs)
for i in range(0,len(rs)):
    # H_00 is just E_g
    Htot[0,0] = Eg_spline(rs[i])
    Htot[1,1] = Eg_spline(rs[i]) + om
    Htot[1,2] = E_au * mu_spline(rs[i]) 
    Htot[2,1] = E_au * mu_spline(rs[i]) 
    Htot[2,2] = Ee_spline(rs[i])


    vals, vecs = LA.eigh(Htot)
    pl_1[i] = np.real(vals[0])
    pl_2[i] = np.real(vals[1])
    pl_3[i] = np.real(vals[2])





In [ ]:
plt.plot(rs, Eg_spline(rs)+om, 'red')
#plt.plot(rs, pl_1, 'ro-')
plt.plot(rs, Ee_spline(rs), 'blue')
#plt.plot(rs, pl_2, 'b--')
#plt.plot(rs, pl_3, 'g--')
plt.show()

In [ ]:
# function to plot
def lp(amplitude, omega, r_val, Eg_spline, Ee_spline, mu_spline):
    H = np.zeros((3,3))
    lp_vals = np.zeros_like(r_val)
    for i in range(0, len(r_val)):
        H[0,0] = Eg_spline(r_val[i])
        H[1,1] = Eg_spline(r_val[i]) + omega
        H[1,2] = amplitude * mu_spline(r_val[i])
        H[2,1] = amplitude * mu_spline(r_val[i])
        H[2,2] = Ee_spline(r_val[i])
        vals, vecs = LA.eigh(H)
        lp_vals[i] = vals[1]
        
    return lp_vals

# function to plot
def up(amplitude, omega, r_val, Eg_spline, Ee_spline, mu_spline):
    H = np.zeros((3,3))
    up_vals = np.zeros_like(r_val)
    for i in range(0, len(r_val)):
        H[0,0] = Eg_spline(r_val[i])
        H[1,1] = Eg_spline(r_val[i]) + omega
        H[1,2] = amplitude * mu_spline(r_val[i])
        H[2,1] = amplitude * mu_spline(r_val[i])
        H[2,2] = Ee_spline(r_val[i])
        vals, vecs = LA.eigh(H)
        up_vals[i] = vals[2]
        
    return up_vals

In [ ]:
print(Eg_spline(rs))
print(Ee_spline(rs))
print(mu_spline(rs))

pl_2 = lp(0.003, om, rs, Eg_spline, Ee_spline, mu_spline)
pl_3 = up(0.003, om, rs, Eg_spline, Ee_spline, mu_spline)

print(rs)
print(pl_2)
print(pl_3)

In [ ]:
plt.plot(rs, pl_2, 'b--')
plt.plot(rs, pl_3, 'g--')
plt.show()
#plt.plot(rs, mu_spline(rs))
#plt.show()
#print(0.005 / 2.0)

### Questions:
    
1.  What happens to the curves as you increase the magnitude of the electric field (`E_au`)
?



2. What happens to the curves as you decrease the magnitude of the electric field (`E_au`)?



3. What are the implications as far as chemical reactions are concerned on being able to change
the polaritonic curves through features of the electric field itself?

In [ ]:
%matplotlib widget
import ipympl
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
# Create a slider widget and a widget to hold the plot.
slider = widgets.FloatSlider(description = r'$E_{field}$',
                            value = 0.0,
                            min = 0.0,
                            max = 0.5)
# We can use HTML styles to control the appearance of the widget.
plot_widget = widgets.Output(layout = {'width':'100%', 'border': '1px solid black'})

In [ ]:
# function to plot
def lp(amplitude, r_val, Eg_spline):
    return Eg_spline(r_val)

    
    
def up(amplitude, r_val, Ee_spline):
    return Ee_spline(r_val)
    


In [ ]:
# function to plot
def lp(amplitude, omega, r_val, Eg_spline, Ee_spline, mu_spline):
    H = np.zeros((3,3))
    lp_vals = np.zeros_like(r_val)
    for i in range(0, len(r_val)):
        H[0,0] = Eg_spline(r_val[i])
        H[1,1] = Eg_spline(r_val[i]) + omega
        H[1,2] = amplitude * mu_spline(r_val[i])
        H[2,1] = amplitude * mu_spline(r_val[i])
        H[2,2] = Ee_spline(r_val[i])
        vals, vecs = LA.eigh(H)
        lp_vals[i] = vals[1]
        
    return lp_vals

# function to plot
def up(amplitude, omega, r_val, Eg_spline, Ee_spline, mu_spline):
    H = np.zeros((3,3))
    up_vals = np.zeros_like(r_val)
    for i in range(0, len(r_val)):
        H[0,0] = Eg_spline(r_val[i])
        H[1,1] = Eg_spline(r_val[i]) + omega
        H[1,2] = amplitude * mu_spline(r_val[i])
        H[2,1] = amplitude * mu_spline(r_val[i])
        H[2,2] = Ee_spline(r_val[i])
        vals, vecs = LA.eigh(H)
        up_vals[i] = vals[2]
        
    return up_vals

In [ ]:
# Create the plot. 

x = np.linspace(1.25, 3.5, 50)

# Turn off interactive mode before creating the 
# plot so it doesn't display too early.
plt.ioff() # Turn off interactive mode
fig, ax = plt.subplots(constrained_layout = True, figsize=[5,4]);
plt.ion() # Turn on interactive mode

line1, = ax.plot(rs, slider.value*np.sin(rs));
line2, = ax.plot(rs, slider.value*np.sin(2*rs));
ax.set_ylabel(r'PES')
ax.set_xlabel(r'$r$')

# force the figure to display in the plot_widget
with plot_widget:
    display(fig.canvas)


In [ ]:
# Create a function to re-draw the plot using the value of the slider. It is 
# faster to change the y-data but you can erase the plot and draw a new one.
def update(value):
    '''We can use `slider.value` or `value.new` to get the new slider value.'''
    with plot_widget:
        line1, = ax.plot(rs, slider.value*np.sin(rs))
        line2, = ax.plot(rs, slider.value*np.sin(2*rs))
        fig.canvas.draw()

# Set an observer to call `update` whenever the value changes.
slider.observe(update)

In [ ]:
widgets.VBox([plot_widget, slider])

<a id=’Derivation’></a>
### Derivation